# Entendo os Geradores do Python

Vamos supor que tu esteja no mercado com uma lista de compras. Para cada item da lista, tu coloca ele no carrinho e risca da lista. Tu consegue carregar a lista nas mãos até um dado tamanho, mas e se a lista tiver 10000 itens? E se ela for infinita?

A memória do computador é limitada, ali vão caber listas de alguns megas ou alguns gigas, mas listas muito grandes ou que você não sabe o tamanho podem quebrar a memória. Estruturas como arrays, listas e dicionários ficam armazenadas na memória, e por isso são limitadas pelo espaço disponível. Um gerador fica armazenado em outro lugar (em disco ou em banco, por exemplo) e por isso só coloca na memória o elemento que está sendo trabalhado no momento.

## Geradores
Vamos análisar um problema que vai nos ajudar a entender os geradores.

Rode por sua conta e risco
i = 0
while True:
    print(i)
    i+=1

Temos um loop infinito que a cada iteração vai incrementar `i` em 1.

In [12]:
def run_till_end_1():
    i = 0
    while True:
        return i
        i+=1

Agora, temos uma função. Quando rodarmos `run_till_end()` ela vai retornar 0 e encerrar. Mesmo com o loop infinito ela encerra com o return , mas nós queremos todos os valores!

In [11]:
run_till_end_1()

0

In [13]:
def run_till_end_2():
    i = 0
    while True:
        yield i
        i+=1

Usando o `yield` para retornar os valores, cada vez que chamarmos a função ele vai andar um passo. A cada chamada ela inicia a execução do ponto onde parou na chamada anterior. A grosso modo, funções com yield são chamadas de geradores.

In [15]:
run = run_till_end_2()
print(type(run))
print(next(run))  # 0
print(next(run))  # 1

<class 'generator'>
0
1


Se rodarmos esse código, o tipo da função será generator. O primeiro print vai retornar 0 e o segundo vai retornar 1. 

O mesmo pode ser feito com listas.

In [16]:
def run_list():
    lst = ['foo', 'baz', 'bar']
    for i in lst:
        yield i

In [17]:
run = run_list()
print(next(run))  # foo
print(next(run))  # baz

foo
baz


Mas e se chamarmos next 4 vezes na run_list ?

Na quarta chamada vamos ter o erro StopIteration . Essa exeção é jogada sempre que o gerador tiver iterado por todos os itens.

In [18]:
print(next(run))  # bar
print(next(run))  # ERRO!

bar


StopIteration: 

Existe outra maneira de criar geradores, quase como um list comprehension, o generator comprehension.

In [19]:
quadrados_list = [n**2 for n in range(5)]  # cria uma lista
quadrados_gene = (n**2 for n in range(5))  # cria um gerador

Quando você roda esse código, quadrados_list será uma lista e quadrados_gene será um gerador.

In [20]:
quadrados_list

[0, 1, 4, 9, 16]

In [21]:
quadrados_gene

<generator object <genexpr> at 0x7f9c5c048ed0>

### Lembrando:
List comprehension usam colchetes `[ ]`
Generator comprehension usam parenteses `( )`

## Análisando a memória e velocidade
Vamos análisar o uso de memória e o desempenho de listas e geradores. Para isso vamos criar os quadrados de 1 milhão de números.

In [22]:
quadrados_list = [n**2 for n in range(1000000)]
quadrados_gene = (n**2 for n in range(1000000))

In [23]:
import sys

In [24]:
sys.getsizeof(quadrados_list)

8697472

In [25]:
sys.getsizeof(quadrados_gene)

128

No quesito memória os geradores ganham com tranquilidade, a lista ocupou 8697472 bytes enquanto o gerador apenas 128.

In [26]:
import cProfile

In [27]:
cProfile.run('sum(quadrados_list)')

         4 function calls in 0.033 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.033    0.033 <string>:1(<module>)
        1    0.000    0.000    0.033    0.033 {built-in method builtins.exec}
        1    0.033    0.033    0.033    0.033 {built-in method builtins.sum}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [28]:
cProfile.run('sum(quadrados_gene)')

         1000005 function calls in 0.558 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  1000001    0.420    0.000    0.420    0.000 <ipython-input-22-1861f011c63a>:2(<genexpr>)
        1    0.000    0.000    0.558    0.558 <string>:1(<module>)
        1    0.000    0.000    0.558    0.558 {built-in method builtins.exec}
        1    0.138    0.138    0.558    0.558 {built-in method builtins.sum}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




Como você pode ver, em termos de desempenho as coisas se invertem. Como a lista já está armazenada em memória, ela acaba sendo mais rápida (0,009 segundos) do que o gerador (0,347 segundos).

## Concluindo
Demos uma olhada nos geradores (ou generators) do Python. Vimos como eles se comportam e com podem ser usados para iterar sobre listas muito grandes ou que não sabemos o tamanho total.

Além dos exemplos que usamos aqui, os generators podem ser usados também para iterarmos sobre arquivos e bancos de dados.